In [1]:
import pyarrow as pa
import pyarrow.parquet as pq
import pyarrow.compute as pc
import pyarrow.dataset as ds
import pandas as pd
import duckdb
import numpy as np
import altair as alt
import json
from urllib.request import urlopen
import time
import os
import plotly.express as px

In [2]:
poet = pd.read_csv('../../CCGIV/datasets/WomenWriting/poet.csv')
poem = pd.read_csv('../../CCGIV/datasets/WomenWriting/poem.csv')
poempoetlinks = pd.read_csv('../../CCGIV/datasets/WomenWriting/poempoetlinks.csv')
subwork = pd.read_csv('../../CCGIV/datasets/WomenWriting/subwork.csv')
subworkpoemlinks = pd.read_csv('../../CCGIV/datasets/WomenWriting/subworkpoemlinks.csv')
subworkpoetlinks = pd.read_csv('../../CCGIV/datasets/WomenWriting/subworkpoetlinks.csv')
work = pd.read_csv('../../CCGIV/datasets/WomenWriting/work.csv')
workpoemlinks = pd.read_csv('../../CCGIV/datasets/WomenWriting/workpoemlinks.csv')
workpoetlinks = pd.read_csv('../../CCGIV/datasets/WomenWriting/workpoetlinks.csv')
poetregionlinks = pd.read_csv('../../CCGIV/datasets/WomenWriting/poetregionlinks.csv')
relation = pd.read_csv('../../CCGIV/datasets/WomenWriting/poetregionlinks.csv')

In [ ]:
work = work[work['worktype'] == '別集']
work

In [ ]:
workpoetlinks = workpoetlinks[workpoetlinks['workID'].isin(work['workID'])]
workpoetlinks

In [ ]:
workpoemlinks = workpoemlinks[workpoemlinks['workID'].isin(work['workID'])]
workpoemlinks

In [ ]:
poet = poet[poet['poetID'].isin(workpoetlinks['poetID'])]
# poet = poet.drop_duplicates(subset='poetID')

poet

In [ ]:
# poem = poem.merge(workpoemlinks, on='poemID',how='inner')
poem = poem[poem['poemID'].isin(workpoemlinks['poemID'])]

# poem = poem.drop_duplicates(subset='poemID')

poem

In [ ]:
con = duckdb.connect()
con.register('poet', poet)
con.register('poem', poem)
con.register('poempoetlinks', poempoetlinks)
con.register('subwork', subwork)
con.register('subworkpoemlinks', subworkpoemlinks)
con.register('subworkpoetlinks', subworkpoetlinks)
con.register('work', work)
con.register('workpoemlinks', workpoemlinks)
con.register('workpoetlinks', workpoetlinks)
con.register('poetregionlinks', poetregionlinks)
con.register('relation', relation)

In [ ]:
ticiSQL = '''
SELECT workID,count(DISTINCT poetID) as ticicount
FROM workpoetlinks 
WHERE role IN ('題辭')
Group By workID
ORDER BY ticicount DESC
'''
tici = duckdb.query(ticiSQL).df()
tici.set_index('workID', inplace=True)

xuSQL = '''
SELECT workID,count(DISTINCT poetID) as xucount
FROM workpoetlinks 
WHERE role IN ('序作者')
Group By workID
ORDER BY xucount DESC
'''
xu = duckdb.query(xuSQL).df()
xu.set_index('workID', inplace=True)

baSQL = '''
SELECT workID,count(DISTINCT poetID) as bacount
FROM workpoetlinks 
WHERE role IN ('跋作者')
Group By workID
ORDER BY bacount DESC
'''
ba = duckdb.query(baSQL).df()
ba.set_index('workID', inplace=True)

from functools import reduce

TiciXuBa = reduce(lambda left, right: pd.merge(left, right, on='workID', how='outer'), [tici, xu, ba]).fillna(0)
TiciXuBa

includedWorkSQL = '''
SELECT workID,count(poemID) as includedcount
FROM workpoemlinks 
Group By workID
ORDER BY includedcount DESC
'''
includedWork = duckdb.query(includedWorkSQL).df()
includedWork.set_index('workID', inplace=True)

jiRawData = reduce(lambda left, right: pd.merge(left, right, on='workID', how='outer'),
                   [tici, xu, ba, includedWork]).fillna(0)

ji_combine_normalize = jiRawData.copy()

all_counts = np.concatenate(
    (ji_combine_normalize['ticicount'], ji_combine_normalize['xucount'], ji_combine_normalize['bacount']))

min_val = all_counts.min()
max_val = all_counts.max()
ji_combine_normalize['normalized_bacount'] = (ji_combine_normalize['bacount'] - min_val) / (max_val - min_val)
ji_combine_normalize['normalized_ticicount'] = (ji_combine_normalize['ticicount'] - min_val) / (max_val - min_val)
ji_combine_normalize['normalized_xucount'] = (ji_combine_normalize['xucount'] - min_val) / (max_val - min_val)


def min_max_normalize_ori(series):
    return (series - series.min()) / (series.max() - series.min())


# 收录的作品数单独归一化
if ji_combine_normalize['includedcount'].dtype in ['float64', 'int64']:  # 检查数据类型是否为数值类型
    ji_combine_normalize['includedcount'] = min_max_normalize_ori(ji_combine_normalize['includedcount'])

xuweight = 0.25
baweight = 0.25
ticiweight = 0.25
includedweight = 0.25

ji_combine = ji_combine_normalize.copy()

ji_combine['totalWeight'] = xuweight * ji_combine['normalized_xucount'] + baweight * ji_combine[
    'normalized_bacount'] + ticiweight * ji_combine['normalized_ticicount'] + includedweight * ji_combine[
                                'includedcount']

ji_combine

In [ ]:
PoetParticipateWorkSQL = '''SELECT poetID, COUNT(DISTINCT workID) AS participate_count
    FROM workpoetlinks
    GROUP BY poetID
    ORDER BY participate_count DESC'''

PoetParticipateWork = duckdb.query(PoetParticipateWorkSQL).df()
PoetParticipateWork.set_index('poetID', inplace=True)

xiangzanauthorSQL = '''
SELECT poempoetlinks.poetID,COUNT(DISTINCT poem.poemID) AS xiangzanauthor_count
FROM poem,poempoetlinks
WHERE poem.GenreHZ='文﹕像贊' AND poem.poemID = poempoetlinks.poemID
GROUP BY poempoetlinks.poetID
ORDER BY xiangzanauthor_count
'''
xiangzanauthor = duckdb.query(xiangzanauthorSQL).df()
xiangzanauthor.set_index('poetID', inplace=True)

bexiangzanSQL = '''
SELECT poetassubjectID AS poetID,COUNT(DISTINCT poemID) AS bexiangzansubjectCount
FROM poem
WHERE poem.GenreHZ='文﹕像贊' AND poetassubjectID !=0
GROUP BY poetassubjectID
'''
bexiangzan = duckdb.query(bexiangzanSQL).df()
bexiangzan.set_index('poetID', inplace=True)

discussedSQL = '''
SELECT poetassubjectID AS poetID, COUNT(DISTINCT poemID) AS discussedCount
FROM poem
WHERE GenreHZ!='文﹕像贊' AND poetassubjectID != 0
GROUP BY poetassubjectID
'''
discussed = duckdb.query(discussedSQL).df()
discussed.set_index('poetID', inplace=True)

changheshiSQL = '''
SELECT changheshuxinpoetID AS poetID, COUNT(DISTINCT poemID) AS changheshiCount
FROM poem
WHERE changheshuxinpoetID !=0
GROUP BY changheshuxinpoetID
'''
changheshi = duckdb.query(changheshiSQL).df()
changheshi.set_index('poetID', inplace=True)

poetRawData = reduce(lambda left, right: pd.merge(left, right, on='poetID', how='outer'),
                     [PoetParticipateWork, xiangzanauthor, bexiangzan, discussed, changheshi]).fillna(0)

poet_log_normalize = poetRawData.copy()

for column in poet_log_normalize.columns:
    if poet_log_normalize[column].dtype in ['float64', 'int64']:
        # 对数据加1以避免对0取对数的问题
        max_val = poet_log_normalize[column].max()
        if max_val > 0:
            # 应用对数归一化
            poet_log_normalize[column] = np.log(poet_log_normalize[column] + 1) / np.log(max_val + 1)

participateWeight = 0.4
writeXZWeight = 0.3
inXZWeight = 0.1
bediscussedWeight = 0.2
changheWeight = 0.1

poet_log = poet_log_normalize.copy()

poet_log['totalWeight'] = participateWeight * poet_log['participate_count'] + writeXZWeight * poet_log[
    'xiangzanauthor_count'] + inXZWeight * poet_log['bexiangzansubjectCount'] + bediscussedWeight * poet_log[
                              'discussedCount'] + changheWeight * poet_log['changheshiCount']

poet_log

In [ ]:
workpoetlinks.groupby(['workID'])
workpoetlinks

In [ ]:
workpoetinfo = workpoetlinks.copy()
workpoetinfo = workpoetinfo.merge(poet, on='poetID', how='inner')
workpoetinfo = workpoetinfo.merge(work, on='workID', how='inner')
workpoetinfo

In [ ]:
worktogether = workpoetinfo[['workID', 'TitleHZ', 'poetID', 'NameHZ', 'role', 'Sex']]
worktogether

In [ ]:
role_priority = ['主要作者', '作者', '其他作者']


def find_primary_author(group):
    # 移除分组键 `workID`，以避免未来版本的警告
    group = group.drop(columns='workID')
    for role in role_priority:
        primary_authors = group[group['role'] == role]
        if not primary_authors.empty:
            return primary_authors.iloc[0]
    return group.iloc[0]


# 使用 groupby 并应用函数
primary_authors = (
    worktogether.groupby('workID', group_keys=False)
    .apply(find_primary_author)
)

# 将分组键 `workID` 重新设为索引
primary_authors['workID'] = primary_authors.index
primary_authors = primary_authors.set_index('workID')

# 转换为字典
primary_authors_dict = primary_authors['NameHZ'].to_dict()
primary_authors_dict

In [ ]:
primary_authors_dict.get(9, None)

In [ ]:
import requests
from zhconv import convert


def find_person_with_specific_source(data, target_source="明清婦女著作數據庫"):
    """在 JSON 数据中查找包含特定来源的 Person 项，若无匹配则返回第一个 Person 项"""

    # 安全地获取 'PersonInfo'，确保它是字典
    person_info = data.get("Package", {}).get("PersonAuthority", {}).get("PersonInfo", {})

    if not isinstance(person_info, dict):
        print("Warning: 'PersonInfo' 不是字典或为空字符串。")
        return None

    # 获取 'Person' 数据
    persons = person_info.get("Person", [])

    # 如果 `persons` 是字典，将其转换为列表以统一处理
    if isinstance(persons, dict):
        persons = [persons]

    first_person = None

    # 遍历每个 Person，查找包含目标来源的项
    for person in persons:
        if first_person is None:
            first_person = person
        # 检查 `PersonSources` 是否存在并且非空
        if isinstance(person, dict) and "PersonSources" in person and person["PersonSources"]:
            person_sources = person["PersonSources"]
            if isinstance(person_sources, dict) and "Source" in person_sources and person_sources["Source"]:
                person_sources = person_sources["Source"]

            # 如果 `person_sources` 是单个字典，将其转换为列表
            if isinstance(person_sources, dict):
                person_sources = [person_sources]

            # 检查是否包含目标来源
            for source in person_sources:
                if isinstance(source, dict) and source.get("Source") == target_source:
                    return person  # 返回包含目标来源的 Person 数据

    # 若未找到目标来源的 Person，则返回第一个 Person
    return first_person


def find_kinship(data):
    # 如果当前层级是字典并且包含 "Kinship"，直接返回
    if isinstance(data, dict):
        if "Kinship" in data:
            return data["Kinship"]
        # 递归地在子层级中查找
        for key, value in data.items():
            result = find_kinship(value)
            if result is not None:
                return result
    # 如果当前层级是列表，检查每个元素
    elif isinstance(data, list):
        for item in data:
            result = find_kinship(item)
            if result is not None:
                return result
    return None


# 获取每行 poet 与主要作者的关系 
def get_relationship(row):
    # print(row)
    primary_author = primary_authors_dict.get(row['workID'], None)
    if primary_author == row['NameHZ']:
        if row['role'] == '序作者':
            return '自序'
        return 'own'
    # print(primary_author)
    if not primary_author:
        return None  # 没有主要作者则返回 None

    # 发送请求到 CBDB API
    url = f'https://cbdb.fas.harvard.edu/cbdbapi/person.php?name={primary_author}&o=json'
    # print(url)
    try:
        response = requests.get(url)
        response.raise_for_status()  # 检查请求是否成功
        rawData = response.json()
        person_with_target_source = find_person_with_specific_source(rawData)
        # print(person_with_target_source)
        if not person_with_target_source:
            return None
        # print(rawData)
        # if not (rawData.get('Package') and
        # rawData['Package'].get('PersonAuthority') and
        # rawData['Package']['PersonAuthority'].get('PersonInfo') and
        # rawData['Package']['PersonAuthority']['PersonInfo'].get('Person')):
        #     return "None"

        # person_kinship_info = response.json()
        # 解析响应数据
        # person_kinship_info = rawData['Package']['PersonAuthority']['PersonInfo']['Person']

        # if person_kinship_info['package']:
        #     person_kinship_info = person_kinship_info['package']
        #     if person_kinship_info['PersonAuthority']:
        #         person_kinship_info = person_kinship_info['PersonAuthority']
        #         if person_kinship_info['PersonInfo']:
        #             PersonInfo =  person_kinship_info['PersonInfo']
        person_kinship_info = find_kinship(person_with_target_source)
        print(person_kinship_info)
        if person_kinship_info is None:
            return None
        # print(person_kinship_info)
        # if isinstance(person_kinship_info, list):
        #     person_kinship_info = person_kinship_info[0]
        # 
        # person_kinship_info = person_kinship_info.get("PersonKinshipInfo", {}).get("Kinship", [])

        # print(person_kinship_info)

        if isinstance(person_kinship_info, dict):
            person_kinship_info = [person_kinship_info]

        # 获取主要作者及 kinship 列表中的所有姓氏
        primary_surname = convert(primary_author[0], 'zh-cn')
        kinship_surnames = {primary_surname}

        # 遍历列表中的每个 kin 信息
        for kin in person_kinship_info:
            kin_name = kin.get("KinPersonName")
            if kin_name:
                kinship_surnames.add(convert(kin_name[0], 'zh-cn'))
            # print(kin.get("KinPersonName"), row['NameHZ'])  # 调试输出
            # 使用简体化后进行比较
            if convert(kin.get("KinPersonName"), 'zh-cn') == convert(row['NameHZ'], 'zh-cn'):
                return kin.get("KinRelName")  # 找到匹配关系时返回 KinRelName

        if convert(row['NameHZ'][0], 'zh-cn') in kinship_surnames:
            return 'suspicious'

        return None


    except requests.RequestException as e:
        print(f"Failed to retrieve data from the API for {primary_author}: {e}")
        return None



In [ ]:
# 创建 DataFrame 切片
primary_authors_dict = {k: ('吳繡珠' if v == '吳秀珠' else v) for k, v in primary_authors_dict.items()}

test = worktogether[worktogether["workID"] == 15].copy()  # 使用 .copy() 避免 SettingWithCopyWarning

# 使用 .loc 替换 'NameHZ' 列中的值
test.loc[:, 'NameHZ'] = test['NameHZ'].replace('吳秀珠', '吳繡珠')

# 使用 .loc 给新列 'relation' 赋值
test.loc[:, 'relation'] = test.apply(get_relationship, axis=1)

# 查看结果
print(test)

In [ ]:
worktogetherraltion = worktogether.copy()
# worktogetherraltion['NameHZ'] = worktogetherraltion['NameHZ'].replace('吳秀珠', '吳繡珠')
worktogetherraltion['relation'] = worktogetherraltion.apply(get_relationship, axis=1)
worktogetherraltion

In [ ]:
# 记录work和poet之间的关系，以及poet和poet之间的亲属关系
import json

# 初始化节点和链接列表
nodes = []
links = []

work_nodes = {}
poet_nodes = {}

# 将 NaN 替换为 None
data = worktogetherraltion.replace({np.nan: None})

# Step 1: 构建 is_zixu_dict 字典，记录每个 poet 是否为 自序
is_zixu_dict = {}

# 按 workID 分组，构建 is_zixu 字典
for work_id, group in data.groupby("workID"):
    for _, row in group.iterrows():
        poet_id_str = f"poet_{row['poetID']}"
        # 将 relation 中的 "自序" 标记为 True，不覆盖已经存在的 True 值
        if poet_id_str not in is_zixu_dict:
            is_zixu_dict[poet_id_str] = row['relation'] == "自序"
        elif row['relation'] == "自序":
            is_zixu_dict[poet_id_str] = True

# Step 2: 构建节点和链接列表
for _, row in data.iterrows():
    work_id = f"work_{row['workID']}"
    poet_id = f"poet_{row['poetID']}"

    # 添加 work 节点
    if work_id not in work_nodes:
        work_nodes[work_id] = {
            "id": work_id,
            "titleHZ": row['TitleHZ'],
            "group": "work"
        }

    # 添加 poet 节点
    if poet_id not in poet_nodes:
        poet_nodes[poet_id] = {
            "id": poet_id,
            "nameHZ": row['NameHZ'],
            "sex": row['Sex'],
            "is_main_author": primary_authors_dict.get(row['workID']) == row['NameHZ'],
            "is_zixu": is_zixu_dict.get(poet_id, False),  # 使用字典中的值作为 is_zixu 的标记
            "is_suspicious": row['relation'] == "suspicious",
            "group": "poet"
        }

    # 添加 work-poet 链接
    links.append({
        "source": work_id,
        "target": poet_id,
        "group": "workpoet"
    })

# 添加 poet-poet 链接（非空、非自序、非 suspicious 的关系）
for work_id, group in data.groupby("workID"):
    valid_poets = group[
        (group['relation'].notnull()) & (group['relation'] != 'own') & (group['relation'] != 'suspicious')]

    # 按 poet 顺序将节点依次相连形成链状
    for i in range(len(valid_poets) - 1):
        source_poet = f"poet_{valid_poets.iloc[i]['poetID']}"
        target_poet = f"poet_{valid_poets.iloc[i + 1]['poetID']}"
        links.append({
            "source": source_poet,
            "target": target_poet,
            "relation": valid_poets.iloc[i + 1]['relation'],
            "group": "poetpoet"
        })

# Step 3: 组合 work_nodes 和 poet_nodes 到 nodes 列表中
nodes.extend(work_nodes.values())
nodes.extend(poet_nodes.values())

# Step 4: 检查 links 中引用的节点是否都在 nodes 中，如果缺失则补充
work_ids_in_links = {link['source'] for link in links if link['source'].startswith("work_")}
work_ids_in_links.update(link['target'] for link in links if link['target'].startswith("work_"))
poet_ids_in_links = {link['source'] for link in links if link['source'].startswith("poet_")}
poet_ids_in_links.update(link['target'] for link in links if link['target'].startswith("poet_"))

# 确保所有 workID 都在节点中
for work_id in work_ids_in_links:
    if not any(node['id'] == work_id for node in nodes):
        nodes.append({"id": work_id, "group": "work", "titleHZ": "Unknown Title"})  # 占位符

# 确保所有 poetID 都在节点中
for poet_id in poet_ids_in_links:
    if not any(node['id'] == poet_id for node in nodes):
        nodes.append({"id": poet_id, "group": "poet", "nameHZ": "Unknown Name"})  # 占位符

# Step 5: 保存为 JSON 文件
output = {"nodes": nodes, "links": links}
with open("output_fixed.json", "w", encoding="utf-8") as f:
    json.dump(output, f, ensure_ascii=False, indent=4)

print("Output saved to output_fixed.json")

In [ ]:
is_zixu_dict

In [ ]:
# worktogetherraltionnotnone = worktogetherraltion[worktogetherraltion['relation'] != 'None']
# worktogetherraltionnotnone = worktogetherraltionnotnone[worktogetherraltionnotnone['relation'] != 'own']
worktogetherraltionnotnone = worktogetherraltion[
    worktogetherraltion['relation'].notna() & (worktogetherraltion['relation'] != "None")]
worktogetherraltionnotnone

In [ ]:
worktogetherraltionnotnone['relation'].unique()

In [ ]:
worktogetherraltionnotnone_filtered = worktogetherraltionnotnone[worktogetherraltionnotnone['relation'].notna()]
worktogetherraltionnotnone_filtered

In [ ]:
worktogetherraltionnotnone_filtered['workID'].value_counts()

In [ ]:
worktogetherraltionnotnone_filtered['poetID'].value_counts()

In [ ]:
owndata = worktogetherraltionnotnone_filtered[worktogetherraltionnotnone['relation'] == 'own']

In [ ]:
owndata.value_counts('role')

In [ ]:
import pandas as pd
import json


def classify_relation(rel):
    if rel is None:
        return 'unknown'
    elif rel == 'suspicious':
        return 'suspicious'
    else:
        return 'relative'


# 假设df是已经加载的DataFrame
df = worktogetherraltion.copy()

# 应用分类函数
df['relation'] = df['relation'].apply(classify_relation)

df.fillna('unknown', inplace=True)  # 替换 NaN 为 'unknown'

nodes = df[['poetID', 'NameHZ', 'Sex']].drop_duplicates().to_dict(orient='records')
links = []

for work_id, group in df.groupby('workID'):
    poets = group['poetID'].tolist()
    # 循环以形成闭环
    for i in range(len(poets)):
        next_poet = poets[(i + 1) % len(poets)]  # 使用模运算确保闭环
        current_poet = poets[i]

        relation_current = group[group['poetID'] == current_poet]['relation'].iloc[0]
        relation_next = group[group['poetID'] == next_poet]['relation'].iloc[0]

        # 根据两者关系决定链接的关系
        if 'unknown' in [relation_current, relation_next]:
            final_relation = 'unknown'
        elif 'suspicious' in [relation_current, relation_next]:
            final_relation = 'suspicious'
        else:
            final_relation = 'relative'

        link = {
            'source': int(current_poet),
            'target': int(next_poet),
            'workID': int(group['workID'].iloc[0]),
            'workTitle': group['TitleHZ'].iloc[0],
            'relation': final_relation
        }
        links.append(link)

graph = {
    'nodes': nodes,
    'links': links
}

# 保存JSON数据
with open("graph_json.json", "w", encoding="utf-8") as f:
    json.dump(graph, f, ensure_ascii=False, indent=4)

print("Output saved to graph_json.json")

In [ ]:
len(poet[poet['EndYear'] != 0])

In [ ]:
len(poet[poet['StartYear'] != 0])

In [ ]:
len(work[work['PubStartYear'] != 0])

In [ ]:
len(work[work['PubEndYear'] != 0])

In [ ]:
work['PubEndYear'].max()

In [ ]:
work['PubStartYear'].min()

In [ ]:
work[work['PubStartYear'] != 0]['PubStartYear'].min()

In [ ]:
yearcal = work.copy()
yearcal['span'] = yearcal['PubEndYear'] - yearcal['PubStartYear']
yearcal['span'].value_counts()

In [ ]:
import pandas as pd
import json

df = worktogetherraltion.copy()
df.fillna('unknown', inplace=True)  # 替换 NaN 为 'unknown'

# 示例 DataFrame 和 primary_authors_dict
# 你需要确保 df 和 primary_authors_dict 已经正确定义

# 初始化节点和链接列表
nodes = []
links = []
added_poet_ids = set()  # 用于跟踪已添加的作者ID

# 遍历每个作品的作者
for work_id, group in df.groupby('workID'):
    # 从字典获取主要作者的名字
    main_author_name = primary_authors_dict.get(work_id)
    if main_author_name is None:
        continue  # 如果找不到主要作者，则跳过这个作品

    # 获取主要作者的行信息
    main_author_df = group[group['NameHZ'] == main_author_name]
    if main_author_df.empty:
        print(f"No main author found for workID {work_id} with name {main_author_name}")
        continue  # 如果找不到对应的主要作者数据，输出错误信息并跳过

    main_author = main_author_df.iloc[0]
    main_node = {'id': int(main_author['poetID']), 'name': main_author['NameHZ']}

    # 添加主要作者节点，检查是否已添加过
    if main_author['poetID'] not in added_poet_ids:
        nodes.append(main_node)
        added_poet_ids.add(main_author['poetID'])

    # 添加其他作者作为节点，并创建链接
    for _, row in group.iterrows():
        if row['NameHZ'] != main_author_name:
            co_author = {'id': row['poetID'], 'name': row['NameHZ']}
            if row['poetID'] not in added_poet_ids:
                nodes.append(co_author)
                added_poet_ids.add(row['poetID'])
            # 链接主要作者和其他作者
            links.append(
                {'source': int(main_author['poetID']), 'target': int(row['poetID']), 'relation': row['relation']})

    # 在其他作者之间创建闭合链
    co_authors = group[group['NameHZ'] != main_author_name]
    if not co_authors.empty:
        first_co_author = co_authors.iloc[0]['poetID']
        last_co_author = None
        for i, row in enumerate(co_authors.iterrows()):
            current_author_id = row[1]['poetID']
            if last_co_author is not None:
                # 创建闭合链接
                links.append({'source': int(last_co_author), 'target': int(current_author_id), 'relation': 'co-author'})
            last_co_author = current_author_id
        # 最后一个和第一个闭合
        if len(co_authors) > 1:
            links.append({'source': int(last_co_author), 'target': int(first_co_author), 'relation': 'co-author'})

# 转换为JSON
graph = {
    'nodes': nodes,
    'links': links
}
# 保存JSON数据
with open("graph_mainauthor.json", "w", encoding="utf-8") as f:
    json.dump(graph, f, ensure_ascii=False, indent=4)

print("Output saved to graph_mainauthor.json")

In [ ]:
duckdb.query(
    "SELECT workID, COUNT(DISTINCT poetID) AS ticicount FROM workpoetlinks WHERE role='題辭' GROUP BY workID").df()

In [ ]:
poet['StartYear'].value_counts()

In [ ]:
poet['EndYear'].value_counts()

In [ ]:
poet[(poet["StartYear"] == 0) & (poet["EndYear"] == 0)]


In [ ]:
work['PubPlaceHZ'].value_counts()

In [ ]:
work['PubStartYear'].value_counts()

In [ ]:
work['PubNameHZ'].value_counts()

In [ ]:
cycle = pd.read_csv('../../CCGIV/datasets/WomenWriting/cycle.csv')
cycle.to_parquet('cycle.parquet')

In [ ]:
poet['LifeSpan'].value_counts()

In [ ]:
workpoemlinks['poemID'].value_counts(sort=True, ascending=False)

In [ ]:
qury = '''
    SELECT 
        wpl.workID,
        work.worktype,
        wpl.poetID,
        wpl.role,
        COUNT(wp.poemID) AS poemCount
    FROM 
        workpoetlinks AS wpl
    LEFT JOIN 
        poempoetlinks AS pp
    ON 
        wpl.poetID = pp.poetID
    LEFT JOIN work ON wpl.workID = work.workID
    LEFT JOIN
            workpoemlinks as wp
    ON pp.poemID = wp.poemID AND wpl.workID = wp.workID
    GROUP BY 
        wpl.workID, wpl.poetID, wpl.role, work.worktype
    ORDER BY 
        wpl.workID, wpl.poetID;
'''
result = duckdb.query(qury).df()
result

In [ ]:
test = workpoemlinks.merge(poempoetlinks, on='poemID', how="outer").merge(poem, on='poemID', how="outer").merge(work,
                                                                                                                on='workID',
                                                                                                                how="outer")
test = test.fillna('unknown')
test1 = test[test['workID'].astype(str) == '9.0']
test1[['workID', 'poetID', 'poemID', 'GenreHZ']]

In [ ]:
filtered_test = test[(test['workID'].astype(str) == '10.0')]
filtered_test

In [ ]:
test.groupby(['workID', 'poetID', 'GenreHZ'])['poemID'].nunique().reset_index(name='genre_count')

In [ ]:
if 'workID' in test.columns and 'GenreHZ' in test.columns and 'poemID' in test.columns:
    # 删除空值，确保统计不受影响
    test_cleaned = test.dropna(subset=['workID', 'GenreHZ', 'poemID', 'worktype'])

    # 确保每个 workID 和 GenreHZ 的 poemID 唯一性，避免重复计数
    test_unique = test_cleaned.drop_duplicates(subset=['workID', 'GenreHZ', 'poemID', 'worktype'])

    # 统计每个 workID 和 GenreHZ 的唯一 poemID 数量
    genre_count = test_unique.groupby(['workID', 'GenreHZ', 'worktype'])['poemID'].nunique().reset_index(name='count')

    # 对每个 workID 找到出现次数最多的 GenreHZ
    top_genre = genre_count.loc[genre_count.groupby('workID')['count'].idxmax()]

    # 统计每个 workID 的 poem 总数（基于唯一 poemID）
    poem_count = test_cleaned.groupby('workID')['poemID'].nunique().reset_index(name='poem_total')

    # 合并结果
    result = top_genre.merge(poem_count, on='workID')

result

In [ ]:
genre_count

In [ ]:
test_unique[test_unique['workID'].astype(str) == '37.0']

In [ ]:
test[test['workID'].astype(str) == '67.0']

In [ ]:
poem[poem['poetassubjectID'] != 0]['GenreHZ'].value_counts()

In [ ]:
poem[(poem['poetassubjectID'] != 0) & (poem['GenreHZ'] == '圖')]

In [ ]:
poem[(poem['poetassubjectID'] != 0) & (poem['GenreHZ'] == '詩')]

In [ ]:
workpoetlinks.groupby('workID')['poetID'].count()

In [ ]:
workpoetlinks['role'].value_counts()

In [21]:
# def min_max_normalize(series):
#     return (series - series.min()) / (series.max() - series.min()) * 100

def min_max_normalize(series):
    return series / series.max()


PoetParticipateWork = duckdb.query(
    "SELECT poetID, COUNT(DISTINCT workID) AS participate_count FROM workpoetlinks GROUP BY poetID ORDER BY "
    "participate_count DESC").df()

xiangzanauthor = duckdb.query(
    "SELECT poempoetlinks.poetID,COUNT(DISTINCT poem.poemID) AS xiangzanauthor_count FROM poem,poempoetlinks "
    "WHERE poem.GenreHZ='文﹕像贊' AND poem.poemID = poempoetlinks.poemID GROUP BY poempoetlinks.poetID ORDER BY "
    "xiangzanauthor_count").df()

bexiangzan = duckdb.query(
    '''SELECT p.poetassubjectID AS poetID, COUNT(DISTINCT p.poemID) AS bexiangzansubjectCount
        FROM poem p
        WHERE p.GenreHZ IN ('文﹕年譜', '文﹕略傳', '文﹕傳') AND p.poetassubjectID != 0
        GROUP BY p.poetassubjectID;''').df()

discussed = duckdb.query(
    '''SELECT p.poetassubjectID AS poetID, COUNT(DISTINCT p.poemID) AS discussedCount
        FROM poem p
        WHERE p.GenreHZ IN ('文﹕詩話', '文﹕詞話', '文﹕案語') AND p.poetassubjectID != 0
        GROUP BY p.poetassubjectID;''').df()

changheshi = duckdb.query(
    "SELECT changheshuxinpoetID AS poetID, COUNT(DISTINCT poemID) AS changheshiCount FROM poem WHERE "
    "changheshuxinpoetID !=0 GROUP BY changheshuxinpoetID").df()

PoetParticipateWork.set_index('poetID', inplace=True)
xiangzanauthor.set_index('poetID', inplace=True)
bexiangzan.set_index('poetID', inplace=True)
discussed.set_index('poetID', inplace=True)
changheshi.set_index('poetID', inplace=True)

# 合并数据
poetRawData = pd.concat([PoetParticipateWork, bexiangzan, discussed], axis=1).fillna(0)
poetRawData = poetRawData[poetRawData['participate_count'] > 0]
poetRawData = poetRawData.loc[
    (poetRawData.index != 6461) & (poetRawData.index != 6467) & (poetRawData.index != 6468)]
# 归一化
for column in poetRawData.columns:
    poetRawData[column] = min_max_normalize(poetRawData[column])

participateWeight = 0.33
writeXZWeight = 0
inXZWeight = 0.33
bediscussedWeight = 0.33
changheWeight = 0
poet_min_max = poetRawData.copy()
poet_min_max['participateCalc'] = participateWeight * poet_min_max['participate_count']
# poet_min_max['writeXZCalc'] = writeXZWeight * poet_min_max['xiangzanauthor_count']
poet_min_max['inXZCalc'] = inXZWeight * poet_min_max['bexiangzansubjectCount']
poet_min_max['bediscussedCalc'] = bediscussedWeight * poet_min_max['discussedCount']
# poet_min_max['changheCalc'] = changheWeight * poet_min_max['changheshiCount']

poet_min_max['totalWeight'] = poet_min_max['participateCalc'] + poet_min_max['inXZCalc'] + poet_min_max[
    'bediscussedCalc']
# ji_min_max['totalWeight'] = xuweight * ji_min_max['xucount'] + baweight * ji_min_max['bacount'] + ticiweight * \
#                             ji_min_max['ticicount'] + includedweight * ji_min_max['includedcount']
poet_min_max_sorted = poet_min_max.sort_values('totalWeight', ascending=False)

poet_min_max_sorted[['participate_count', 'bexiangzansubjectCount', 'discussedCount', 'totalWeight']]

,participate_count,bexiangzansubjectCount,discussedCount,totalWeight
poetID,,,,
911,0.307692,0.2,1.000000,0.497538
199,0.692308,0.3,0.224490,0.401543
43,0.153846,1.0,0.061224,0.400973
202,0.538462,0.5,0.040816,0.356162
167,0.538462,0.4,0.122449,0.350100
...,...,...,...,...
8452,0.076923,0.0,0.000000,0.025385
6211,0.076923,0.0,0.000000,0.025385
6196,0.076923,0.0,0.000000,0.025385


In [22]:
poet_min_max_sorted.reset_index()
max_importance = poet_min_max_sorted['totalWeight'].max()
poet_min_max_sorted['normalized_totalWeight'] = poet_min_max_sorted['totalWeight'] / max_importance
poet_min_max_sorted['ln_normalized_totalWeight'] = np.log(poet_min_max_sorted['normalized_totalWeight'] + 0.01)
columns_to_exclude = ['HuWenKai', 'xuZuoZhe', 'baZuoZhe']
poet['MissingFields'] = poet.drop(columns=columns_to_exclude).isnull().sum(axis=1)

poet_min_max_sorted = poet_min_max_sorted.merge(poet, on='poetID', how="inner")[
    ['poetID', 'NameHZ', 'StartYear', 'EndYear', 'totalWeight', 'normalized_totalWeight', 'ln_normalized_totalWeight',
     'MissingFields']]
# poet_min_max_sorted[['poetID','NameHZ','participate_count','bexiangzansubjectCount','discussedCount']]
poet_min_max_sorted

,poetID,NameHZ,StartYear,EndYear,totalWeight,normalized_totalWeight,ln_normalized_totalWeight,MissingFields
0,911,顧太清,1799,1877,0.497538,1.000000,0.009950,2
1,199,吳藻,1799,1862,0.401543,0.807060,-0.202043,0
2,43,顧若璞,1592,1681,0.400973,0.805914,-0.203446,4
3,202,席佩蘭,1762,1842,0.356162,0.715848,-0.320415,0
4,167,歸懋儀,1762,1832,0.350100,0.703665,-0.337341,2
...,...,...,...,...,...,...,...,...
2762,8452,冒國柱,1740,1820,0.025385,0.051020,-2.796547,8
2763,6211,江芳孫,0,0,0.025385,0.051020,-2.796547,7
2764,6196,黃彭年,1824,1890,0.025385,0.051020,-2.796547,1
2765,6991,楊振鎬,0,0,0.025385,0.051020,-2.796547,8


In [23]:
poet_min_max_sorted['StartYear'].value_counts()

StartYear
0       452
1796    220
1644    170
1821    143
1850    131
       ... 
1727      1
1623      1
1616      1
1582      1
1580      1
Name: count, Length: 258, dtype: int64

In [7]:
poet_min_max_sorted_detail = poet_min_max_sorted.copy()
poet_min_max_sorted_detail = poet_min_max_sorted_detail.merge(poet, on='poetID', how="inner")
poet_min_max_sorted_detail

,poetID,NameHZ_x,StartYear_x,EndYear_x,totalWeight,normalized_totalWeight,ln_normalized_totalWeight,MissingFields_x,NameHZ_y,NamePY,...,StartYear_y,EndYear_y,EthnicGroup,MainWorks,ispoet,HuWenKai,zhuLu,xuZuoZhe,baZuoZhe,MissingFields_y
0,911,顧太清,1799,1877,0.497538,1.000000,0.009950,2,顧太清,Gu Taiqing,...,1799,1877,漢,子春集﹐天游閣集五卷﹐東海漁歌四卷,1,800,清史稿藝文志﹐國朝閨秀正始集﹐風雨樓叢書﹐(沈善寶)名媛詩話﹐漁歌﹐(沈湘佩)名媛詩話﹐（徐...,鈍宦案語﹐況周頤﹐王佳﹐壽森序﹐徐德培﹐慶珍﹐伊既明﹐張潤普﹐梁啟勳題詞,NaN,2
1,199,吳藻,1799,1862,0.401543,0.807060,-0.202043,0,吳藻,Wu Zao,...,1799,1862,漢,《花簾詞一卷》、《香南雪北詞一卷》、《花簾書屋詩詞》、《讀騷圖曲》,1,317,杭州府志﹐國朝閨秀正始續集﹐國朝正雅集﹐小檀欒室彙刻閨秀詞﹐碧城仙館女弟子詩﹐崑山徐氏書目﹐...,張景祁﹐陳文述﹐魏謙升﹐趙慶熺序﹐吳藻自序,NaN,0
2,43,顧若璞,1592,1681,0.400973,0.805914,-0.203446,4,顧若璞,Gu Ruopu,...,1592,1681,漢,《臥月軒集六卷》,1,206,翠樓集﹐宮閨氏籍藝文考略﹐國朝閨閣詩鈔﹐（徐乃昌）閨秀詞鈔﹐國朝閨秀正始集﹐全清詞﹐小黛軒論...,吳本泰序﹐力墨子﹐包鴻嘉小傳﹐馬元調序﹐顧若群總序詩稿序﹐顧若璞自序,NaN,4
3,202,席佩蘭,1762,1842,0.356162,0.715848,-0.320415,0,席佩蘭,Xi Peilan,...,1762,1842,漢,"《長真閣詩集》, 《傍杏樓調琴草》, 《長真閣詩餘一卷》",1,469,"蘇州府志﹐清史稿藝文志﹐擷芳集﹐正始集﹐國朝閨閣詩鈔﹐小檀欒室彙刻百家閨秀詞, 隨園女弟子詩...",袁枚序,NaN,0
4,167,歸懋儀,1762,1832,0.350100,0.703665,-0.337341,2,歸懋儀,Gui Maoyi,...,1762,1832,漢,《繡餘小草一卷》﹐《繡餘續草五卷》﹐《再續草一卷》﹐《三續草一卷》﹐《四續草一卷》﹐《五續草...,1,784,清史稿藝文志﹐松江府續志﹐崑山胡氏書目﹐常昭合志﹐蘇州府志﹐閨籍經眼錄﹐販書偶記澤存書庫書目...,戈載﹐段驤﹐陶澍﹐陳鑾﹐魏文瀛,諸以敦,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2762,6971,劉卿采,0,0,0.025385,0.051020,-2.796547,8,劉卿采,Liu Qingcai,...,0,0,漢,NaN,0,NaN,NaN,NaN,NaN,8
2763,7068,于齊慶,1856,1904,0.025385,0.051020,-2.796547,1,于齊慶,Yu Qiqing,...,1856,1904,漢,《小尋暢樓詩文鈔》,0,NaN,《晚晴簃詩匯·卷一七五》《清人詩文集總目提要》(1874頁),NaN,NaN,1
2764,5788,武麟仁,1821,1908,0.025385,0.051020,-2.796547,7,武麟仁,Wu Linren,...,1821,1908,漢,NaN,0,NaN,NaN,NaN,NaN,7
2765,6707,趙掌宣,1796,1820,0.025385,0.051020,-2.796547,7,趙掌宣,Zhao Zhangxuan,...,1796,1820,漢,NaN,0,NaN,NaN,NaN,NaN,7


In [11]:
poem_filter = poempoetlinks[poempoetlinks['poetID'].isin(poet_min_max_sorted_detail['poetID'])]
poem_filter['poemID'].value_counts()

poemID
91773    4
91768    4
91771    4
89719    4
89695    3
        ..
35357    1
35358    1
35359    1
35360    1
97257    1
Name: count, Length: 74015, dtype: int64

In [13]:
poetregionlinks[poetregionlinks['poetID'].isin(poet_min_max_sorted_detail['poetID'])]['regionID'].value_counts()

regionID
110     102
10       82
64       47
18       44
48       43
       ... 
1377      1
1383      1
1387      1
1386      1
1441      1
Name: count, Length: 456, dtype: int64

In [93]:
alt.data_transformers.enable("vegafusion")
poet_min_max_sorted_heatmap = poet_min_max_sorted.copy()
poet_min_max_sorted_heatmap['StartYear'] = poet_min_max_sorted_heatmap['StartYear'].replace(0, "missing")

poet_min_max_sorted_heatmap['ln_normalized_totalWeight'] = poet_min_max_sorted_heatmap[
    'ln_normalized_totalWeight'].fillna(0)
# 创建热力图
heatmap = alt.Chart(poet_min_max_sorted_heatmap).mark_rect().encode(
    x=alt.X('StartYear:O', title='Start Year', bin=alt.Bin(step=5), axis=alt.Axis(labelAngle=-45)),  # 横轴分箱
    y=alt.Y('ln_normalized_totalWeight:Q', title='Normalized Total Weight', bin=alt.Bin(step=0.05)),  # 纵轴分箱
    color=alt.Color('count():Q', title='Density'),  # 用颜色显示计数
    tooltip=['count()', 'StartYear', 'ln_normalized_totalWeight']
).properties(
    width=1000,
    height=200,
    title='Heatmap of Start Year vs Normalized Total Weight'
).interactive()

heatmap.display()

alt.Chart(...)

In [74]:
poet_min_max_sorted_heatmap_copy = poet_min_max_sorted_heatmap.copy()
# poet_min_max_sorted_heatmap_copy['StartYear_filled'] = poet_min_max_sorted_heatmap_copy['StartYear'].fillna('Missing')

duplicates = poet_min_max_sorted_heatmap_copy.groupby(['ln_normalized_totalWeight', 'MissingFields']).size()
duplicates
# poet_min_max_sorted_heatmap_copy[poet_min_max_sorted_heatmap_copy['StartYear']=='missing']

ln_normalized_totalWeight  MissingFields
-2.796547                  0                  3
                           1                 91
                           2                 80
                           3                186
                           4                207
                                           ... 
-0.337341                  2                  1
-0.320415                  0                  1
-0.203446                  4                  1
-0.202043                  0                  1
 0.009950                  2                  1
Length: 199, dtype: int64

In [71]:
poet_min_max_sorted_heatmap_copy

,poetID,NameHZ,StartYear,EndYear,totalWeight,normalized_totalWeight,ln_normalized_totalWeight,MissingFields
0,911,顧太清,1799,1877,0.497538,1.000000,0.009950,2
1,199,吳藻,1799,1862,0.401543,0.807060,-0.202043,0
2,43,顧若璞,1592,1681,0.400973,0.805914,-0.203446,4
3,202,席佩蘭,1762,1842,0.356162,0.715848,-0.320415,0
4,167,歸懋儀,1762,1832,0.350100,0.703665,-0.337341,2
...,...,...,...,...,...,...,...,...
2762,373,許庚身,1821,1893,0.025385,0.051020,-2.796547,7
2763,372,楊昌濬,1821,1897,0.025385,0.051020,-2.796547,7
2764,333,姚福增,1796,1850,0.025385,0.051020,-2.796547,7
2765,8408,董恂,1821,1908,0.025385,0.051020,-2.796547,7


In [72]:
duplicates = poet_min_max_sorted.groupby(['ln_normalized_totalWeight', 'MissingFields']).size()
duplicates

ln_normalized_totalWeight  MissingFields
-2.796547                  0                  3
                           1                 91
                           2                 80
                           3                186
                           4                207
                                           ... 
-0.337341                  2                  1
-0.320415                  0                  1
-0.203446                  4                  1
-0.202043                  0                  1
 0.009950                  2                  1
Length: 199, dtype: int64

In [ ]:

role_weights = {
    '主要作者': 0.6, '作者': 0.6, '其他作者': 0.6,
    '題辭': 0.3, '序作者': 0.3, '跋作者': 0.3, '附记作者': 0.3, '凡例作者': 0.3,
    '墓志詺作者': 0.3, '輓詞作者': 0.3, '傳記作者': 0.3, '像赞作者': 0.3, '年譜作者': 0.3,
    '编輯': 0.1, '校閲': 0.1, '校注者': 0.1
}

In [ ]:
workpoet = workpoetlinks.copy()
workpoet['RoleWeight'] = workpoet['role'].map(role_weights)
workpoet

In [ ]:

workpoet = workpoet.merge(poet_min_max_sorted, on='poetID', how='outer')

In [ ]:
workpoet['PoetImportance'] = workpoet['RoleWeight'] * workpoet['ln_normalized_totalWeight']
workpoet

In [ ]:
# 计算每个作品的总重要性
work_importance_total = workpoet.groupby('workID')['PoetImportance'].sum()

# # 计算每个作品的参与诗人数
# work_poet_count = workpoet.groupby('workID')['poetID'].nunique()
# 
# # 计算标准化的工作重要性
# work_importance = work_importance_total / work_poet_count
# work_importance = work_importance.reset_index()
# work_importance.columns = ['workID', 'StandardizedWorkImportance']

# 归一化处理：每个作品的重要性除以所有作品重要性的最大值
max_importance = work_importance_total.max()
normalized_importance = work_importance_total / max_importance
normalized_importance_df = normalized_importance.reset_index()
raw_importance = normalized_importance_df.copy()
normalized_importance_df = normalized_importance_df.merge(work, on='workID', how='outer')[
    ['workID', 'TitleHZ', 'PubStartYear', 'PoetImportance']]
normalized_importance_df

In [ ]:
columns_to_exclude = ['Summary', 'UniformTitle', 'VariantTitle']

work['MissingFields'] = work.drop(columns=columns_to_exclude).isnull().sum(axis=1)

work_importance_total_df = raw_importance.merge(work, on='workID', how='outer')[
    ['workID', 'TitleHZ', 'PubStartYear', 'PubEndYear', 'PoetImportance', 'MissingFields']]

# 将 X=0 的点放置在 X=1600 的左侧
work_importance_total_df['AdjustedPubStartYear'] = work_importance_total_df['PubStartYear'].apply(
    lambda x: 1590 if x == 0.0 else x
)

# 对数放大 Importance 值
offset = 0.01  # 偏移量
work_importance_total_df['LnPoetImportance'] = np.log(work_importance_total_df['PoetImportance'] + offset)

# 拉大差距：通过指数放大或归一化处理
scaling_factor = 5  # 放大因子
work_importance_total_df['ScaledLnPoetImportance'] = work_importance_total_df['LnPoetImportance'] * scaling_factor

import altair as alt

# 绘制散点图，并设置 X 轴刻度
scatter_plot = alt.Chart(work_importance_total_df).mark_circle(
    stroke='white',  # 边框颜色设置为白色
    strokeWidth=1.5  # 边框宽度
).encode(
    x=alt.X(
        'AdjustedPubStartYear:Q',
        title='Publication Start Year',
        scale=alt.Scale(domain=[1600, 1959], nice=False),  # 定义刻度
        axis=alt.Axis(values=[0, 1600, 1700, 1800, 1900, 1959])  # 显示特定刻度
    ),
    y=alt.Y('LnPoetImportance:Q', title='Normalized Poet Importance', scale=alt.Scale(zero=False)),
    tooltip=['workID', 'TitleHZ', 'PubStartYear', 'PoetImportance']
).properties(
    title='Scatter Plot of Publication Year vs. Poet Importance',
    width=100,
    height=400
)

# 保存并展示图表
scatter_plot.show()

In [ ]:
work_importance_total_df['aggregate_value'] = work_importance_total_df['PoetImportance'] + work_importance_total_df[
    'MissingFields']
work_importance_total_df

In [ ]:
work_importance_total_df_nohead = work_importance_total_df[
    (work_importance_total_df['workID'] != 37.0) & (work_importance_total_df['workID'] != 61.0)]
work_importance_total_df_nohead


In [ ]:
duplicates = work_importance_total_df.groupby(['PoetImportance', 'PubStartYear']).size()
duplicates

In [ ]:
scatter_plot = alt.Chart(poet_min_max_sorted).mark_circle(
    stroke='white',  # 边框颜色设置为白色
    strokeWidth=1.5  # 边框宽度
).encode(
    x=alt.X(
        'StartYear:Q',
        title='Poet Start Year',
        # scale=alt.Scale(domain=[1600, 1959], nice=False),  # 定义刻度
        # axis=alt.Axis(values=[0, 1600, 1700, 1800, 1900, 1959])  # 显示特定刻度
    ),
    y=alt.Y('normalized_totalWeight:Q', title='normalized_totalWeight Importance', scale=alt.Scale(zero=False)),
    tooltip=['poetID', 'NameHZ', 'StartYear', 'normalized_totalWeight']
).properties(
    title='Scatter Plot of Publication Year vs. Poet Importance',
    width=900,
    height=400
)

# 保存并展示图表
scatter_plot.show()

In [ ]:
duplicates = poet_min_max_sorted.groupby(['StartYear', 'normalized_totalWeight']).size()
duplicates

In [ ]:
duplicates = work_importance_total_df.groupby(['LnPoetImportance', 'MissingFields']).size()
duplicates

In [ ]:
from scipy.stats import linregress

alt.data_transformers.enable("vegafusion")

# 共享配置
base = alt.Chart().properties(
    title='Missing Fields Count vs. Reversed Missing Fields',
    width=900,
    height=400
).configure_axis(
    grid=True,  # 显示网格
    domain=True,  # 显示轴线
    domainWidth=1.5  # 调整轴线宽度
)

offset = 0  # 偏移量
work_importance_total_df['LnPoetImportance'] = np.log(work_importance_total_df['PoetImportance'] + offset)

# 移除缺失值
cleaned_df = work_importance_total_df.dropna(subset=['MissingFields', 'LnPoetImportance']).copy()

# 执行回归分析
y = cleaned_df['MissingFields']
x = cleaned_df['LnPoetImportance']
regression_result = linregress(x, y)

# print(f"Regression Slope: {regression_result.slope}")
# print(f"Regression Intercept: {regression_result.intercept}")
# print(f"Correlation Coefficient (r): {regression_result.rvalue}")
# print(f"P-value: {regression_result.pvalue}")
# print(f"Standard Error: {regression_result.stderr}")

# 添加回归预测值到 DataFrame
cleaned_df['RegressionLine'] = regression_result.slope * cleaned_df['LnPoetImportance'] + regression_result.intercept

# 绘制散点图，并设置 X 轴刻度
scatter_plot = alt.Chart(work_importance_total_df).mark_circle(
    stroke='white',  # 边框颜色设置为白色
    strokeWidth=1.5,  # 边框宽度
    opacity=0.2
).encode(
    y=alt.Y(
        'MissingFields:Q',
        title='Missing Fields Count',
        scale=alt.Scale(
            domain=[-max(poet_min_max_sorted["MissingFields"]), max(work_importance_total_df["MissingFields"])]),
        # 包含正负方向
    ),
    x=alt.X('LnPoetImportance:Q', title='ln(Normalized Poet Importance+0.01)'),
    tooltip=['workID', 'TitleHZ', 'PubStartYear', 'PoetImportance']
).properties(
    title='Missing Fields Count vs. Poet Importance',
    width=1200,
    height=400
).interactive()
poet_min_max_sorted['ReversedMissingFields'] = -poet_min_max_sorted['MissingFields']

# 绘制 poet_min_max_sorted 的反向散点图
reversed_plot = alt.Chart(poet_min_max_sorted).mark_circle(color='blue', stroke='white', strokeWidth=1.5,
                                                           opacity=0.5).encode(
    x='ln_normalized_totalWeight:Q',  # 公用横轴
    y=alt.Y(
        'ReversedMissingFields:Q',
        title='Reversed Missing Fields Count'
    ),
    tooltip=['poetID', 'NameHZ', 'ln_normalized_totalWeight', 'MissingFields']
).interactive()

# # 绘制回归线
# regression_line = alt.Chart(cleaned_df).mark_line(color='red').encode(
#     x='LnPoetImportance:Q',
#     y='RegressionLine:Q'
# )

# 合并图表并显示 X 轴在中间
final_chart = alt.layer(
    scatter_plot, reversed_plot
).properties(
    title="Scatter Plot with Central X-Axis"
).configure_axisY(
    grid=True,
    domain=True,
    domainOpacity=0.8
)
# 展示图表
# (scatter_plot + reversed_plot).show()
final_chart.show()

In [ ]:
poet_min_max_sorted['aggregate_value'] = poet_min_max_sorted['MissingFields'] + poet_min_max_sorted[
    'normalized_totalWeight']
poet_min_max_sorted

In [ ]:
duplicates = poet_min_max_sorted.groupby(['aggregate_value', 'StartYear', 'EndYear']).size()
duplicates

In [ ]:
work = pd.read_csv('../../CCGIV/datasets/WomenWriting/work.csv')
work['PubPlaceHZ'] = work['PubPlaceHZ'].fillna("unknown")
work['PubPlaceHZ'].value_counts()

In [ ]:
work['PubNameHZ'] = work['PubNameHZ'].fillna("unknown")
work['PubNameHZ'].value_counts()

In [ ]:
workpoetlinksNew = workpoetlinks.copy()
workpoetlinksNew['workID_prefixed'] = 'work_' + workpoetlinksNew['workID'].astype(str)
workpoetlinksNew['poetID_prefixed'] = 'poet_' + workpoetlinksNew['poetID'].astype(str)
# 提取节点列表（nodes）
work_nodes = workpoetlinksNew['workID_prefixed'].unique()
poet_nodes = workpoetlinksNew['poetID_prefixed'].unique()

# 创建节点的 DataFrame
work_nodes_df = pd.DataFrame({'id': work_nodes, 'group': 'Works'})
poet_nodes_df = pd.DataFrame({'id': poet_nodes, 'group': 'Poets'})

# 合并所有节点
nodes = pd.concat([work_nodes_df, poet_nodes_df], ignore_index=True)

# 提取链接列表（links）
links = workpoetlinksNew.groupby(['workID_prefixed', 'poetID_prefixed']).size().reset_index(name='value')
links.rename(columns={'workID_prefixed': 'source', 'poetID_prefixed': 'target'}, inplace=True)

# 转换为适合桑基图的格式
nodes_list = nodes.to_dict(orient='records')
links_list = links.to_dict(orient='records')


In [ ]:
# 添加带前缀的 ID
workpoetlinksNew = workpoetlinks.copy()
workpoetlinksNew['workID_prefixed'] = 'work_' + workpoetlinksNew['workID'].astype(str)
workpoetlinksNew['poetID_prefixed'] = 'poet_' + workpoetlinksNew['poetID'].astype(str)

# 提取节点列表
work_nodes = workpoetlinksNew['workID_prefixed'].unique()
poet_nodes = workpoetlinksNew['poetID_prefixed'].unique()

# 创建节点 DataFrame
work_nodes_df = pd.DataFrame({'id': work_nodes, 'group': 'Works'})
poet_nodes_df = pd.DataFrame({'id': poet_nodes, 'group': 'Poets'})

# 添加原始 ID 列
work_nodes_df['original_workID'] = work_nodes_df['id'].str.replace('work_', '', regex=False).astype(float)
poet_nodes_df['original_poetID'] = poet_nodes_df['id'].str.replace('poet_', '', regex=False).astype(str)

# 确保数据类型一致
work_importance_total_df['workID'] = work_importance_total_df['workID'].astype(float)
poet_min_max_sorted['poetID'] = poet_min_max_sorted['poetID'].astype(str)

# 检查未匹配的 Works
unmatched_works = work_nodes_df[~work_nodes_df['original_workID'].isin(work_importance_total_df['workID'])]
print("Unmatched Works:")
print(unmatched_works)

# 合并字段
work_nodes_enriched = work_nodes_df.merge(
    work_importance_total_df[['workID', 'PoetImportance', 'MissingFields']],
    left_on='original_workID',
    right_on='workID',
    how='left'
).drop(columns=['original_workID', 'workID'])

poet_nodes_enriched = poet_nodes_df.merge(
    poet_min_max_sorted[['poetID', 'normalized_totalWeight', 'MissingFields']],
    left_on='original_poetID',
    right_on='poetID',
    how='left'
).drop(columns=['original_poetID', 'poetID'])

# 添加 importance 列
work_nodes_enriched['importance'] = work_nodes_enriched['PoetImportance']
poet_nodes_enriched['importance'] = poet_nodes_enriched['normalized_totalWeight']

# 保留需要的列
work_nodes_enriched = work_nodes_enriched[['id', 'group', 'importance', 'MissingFields']]
poet_nodes_enriched = poet_nodes_enriched[['id', 'group', 'importance', 'MissingFields']]

# 合并所有节点
nodes_enriched = pd.concat([work_nodes_enriched, poet_nodes_enriched], ignore_index=True)

# 填充缺失值
nodes_enriched.fillna({
    'importance': 0,
    'MissingFields': 0
}, inplace=True)

# 创建链接列表
links = workpoetlinksNew.groupby(['workID_prefixed', 'poetID_prefixed']).size().reset_index(name='value')
links.rename(columns={'workID_prefixed': 'source', 'poetID_prefixed': 'target'}, inplace=True)

# 转换为适合桑基图的格式
nodes_list = nodes_enriched.to_dict(orient='records')
links_list = links.to_dict(orient='records')

In [ ]:
nodes_list

In [ ]:
links_list

In [ ]:
# 将 nodes 和 links 数据整合为一个字典
output_data = {
    "nodes": nodes_list,
    "links": links_list
}

# 指定保存路径
output_file = 'sankey_data.json'

# 写入 JSON 文件
with open(output_file, 'w') as f:
    json.dump(output_data, f, indent=4)

In [ ]:
workpoetlinks['poetID'].value_counts()

In [3]:
poem['Form'].unique()

array([nan, '七言絕句', '五言律詩', '五言古詩', '七言古詩', '樂府', '五言絕句', '七言排律', '七言律詩',
       '四言詩', '雜言詩', '五言排律', '六言詩', '五言聯句', '騒體', '三言詩', '七言聯句', '未定',
       '其他', '偈', '詞聯句'], dtype=object)